In [2]:
import tkinter as tk
from tkinter import ttk
import requests
import pandas as pd
from datetime import datetime

def american_to_decimal(american_odds):
    if american_odds > 0:
        return (american_odds / 100) + 1
    else:
        return (100 / abs(american_odds)) + 1

def calculate_arbitrage_percentage(odds_A, odds_B):
    decimal_odds_A = american_to_decimal(odds_A)
    decimal_odds_B = american_to_decimal(odds_B)
    arb_percentage = (1 / decimal_odds_A + 1 / decimal_odds_B) * 100
    return round(arb_percentage, 2)

def calculate_bet_amounts(total_bet, odds_A, odds_B, arb_percentage):
    decimal_odds_A = american_to_decimal(odds_A)
    decimal_odds_B = american_to_decimal(odds_B)
    amount_A = (total_bet * (1 / decimal_odds_A)) / (arb_percentage / 100)
    amount_B = (total_bet * (1 / decimal_odds_B)) / (arb_percentage / 100)
    return round(amount_A, 2), round(amount_B, 2)

def create_arbitrage_window(arb_opportunities):
    arb_window = tk.Toplevel()
    arb_window.title("Arbitrage Opportunities")
    for i, arb in enumerate(arb_opportunities):
        tk.Label(arb_window, text=f"Match: {arb['fighter1']} vs {arb['fighter2']}").grid(row=i, column=0)
        tk.Label(arb_window, text=f"Bookmaker: {arb['bookmaker']}").grid(row=i, column=1)
        tk.Label(arb_window, text=f"Arbitrage %: {arb['arb_percentage']}").grid(row=i, column=2)
        tk.Label(arb_window, text=f"Bet on {arb['fighter1']}: ${arb['amount_A']}").grid(row=i, column=3)
        tk.Label(arb_window, text=f"Bet on {arb['fighter2']}: ${arb['amount_B']}").grid(row=i, column=4)

def download_odds(selected_bookmakers, bankroll, arb_opportunities):
    # API URL
    url = "https://api.the-odds-api.com/v4/sports/mma_mixed_martial_arts/odds/?apiKey=0a961c52965144f19df17552e65f101e&regions=us&markets=h2h,spreads&oddsFormat=american"
    response = requests.get(url)
    
    try:
        result = response.json()
    except ValueError:
        print(f"Error decoding API response: {response.text}")
        return pd.DataFrame()
    
    all_matches = []
    for res in result:
        for books in res['bookmakers']:
            if selected_bookmakers != 'All' and books['title'] not in selected_bookmakers:
                continue
            match = {}
            match['event_time'] = datetime.fromisoformat(res['commence_time'].replace('Z', '+00:00')).strftime('%Y-%m-%d %H:%M:%S')
            match['bookmaker'] = books['title']
            match['fighter1'] = books['markets'][0]['outcomes'][0]['name']
            match['fighter2'] = books['markets'][0]['outcomes'][1]['name']
            match['odds_f1'] = books['markets'][0]['outcomes'][0]['price']
            match['odds_f2'] = books['markets'][0]['outcomes'][1]['price']
            
            arb_percentage = calculate_arbitrage_percentage(match['odds_f1'], match['odds_f2'])
            match['arb_percentage'] = arb_percentage
            
            if arb_percentage < 100:
                amount_A, amount_B = calculate_bet_amounts(bankroll, match['odds_f1'], match['odds_f2'], arb_percentage)
                match['amount_A'] = amount_A
                match['amount_B'] = amount_B
                arb_opportunities.append(match)
            
            all_matches.append(match)
    
    df = pd.DataFrame(all_matches)
    if selected_bookmakers == 'All':
        df = df.groupby(['event_time', 'fighter1', 'fighter2']).apply(lambda x: x.loc[x['odds_f1'].idxmax()]).reset_index(drop=True)
    
    top_fights = df.sort_values('event_time').drop_duplicates(subset=['event_time', 'fighter1', 'fighter2']).head(20)
    return top_fights

def refresh_table(tree, selected_bookmakers, arb_opportunities):
    bankroll = float(bankroll_entry.get())
    for row in tree.get_children():
        tree.delete(row)
    df = download_odds(selected_bookmakers, bankroll, arb_opportunities)
    for index, row in df.iterrows():
        tree.insert('', 'end', values=(row['event_time'], row['fighter1'], row['odds_f1'], row['fighter2'], row['odds_f2'], row['bookmaker'], row.get('arb_percentage', 'N/A')))
    
    if arb_opportunities:
        create_arbitrage_window(arb_opportunities)
        arb_opportunities.clear()

def on_bookmaker_select(event):
    selected_bookmakers = bookmaker_var.get()
    refresh_table(tree, selected_bookmakers, arb_opportunities)

if __name__ == "__main__":
    root = tk.Tk()
    root.title("Live Updated UFC Odds")
    
    arb_opportunities = []
    
    tk.Label(root, text="Enter Bankroll:").pack(side=tk.TOP)
    bankroll_entry = tk.Entry(root)
    bankroll_entry.pack(side=tk.TOP)
    bankroll_entry.insert(0, "100")
    
    bookmaker_options = ['All', 'Bovada', 'DraftKings', 'BetRivers', 'BetUS']
    bookmaker_var = tk.StringVar(value='All')
    bookmaker_menu = ttk.Combobox(root, textvariable=bookmaker_var, values=bookmaker_options)
    bookmaker_menu['state'] = 'readonly'
    bookmaker_menu.pack(side=tk.TOP)
    bookmaker_menu.bind("<<ComboboxSelected>>", on_bookmaker_select)
    
    frame = ttk.Frame(root)
    frame.pack(side=tk.TOP, padx=20, pady=20, fill=tk.BOTH, expand=tk.YES)

    scrollbar = ttk.Scrollbar(frame)
    scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

    tree = ttk.Treeview(frame, columns=('Event Time', 'Fighter1', 'Odds1', 'Fighter2', 'Odds2', 'Bookmaker', 'Arbitrage %'), show='headings', yscrollcommand=scrollbar.set)
    tree.heading('Event Time', text='Event Time')
    tree.heading('Fighter1', text='Fighter 1')
    tree.heading('Odds1', text='Odds 1')
    tree.heading('Fighter2', text='Fighter 2')
    tree.heading('Odds2', text='Odds 2')
    tree.heading('Bookmaker', text='Bookmaker')
    tree.heading('Arbitrage %', text='Arbitrage %')
    tree.pack(fill=tk.BOTH, expand=tk.YES)

    scrollbar.config(command=tree.yview)
    
    root.after(1000, refresh_table, tree, 'All', arb_opportunities)
    root.mainloop()
